<a href="https://colab.research.google.com/github/jrebull/NLP/blob/main/Team18_MNA_NLP_semana_10_Actividad_10_3_modelosLLM_IA_Trabajo_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Procesamiento de Lenguaje Natural**

## Maestría en Inteligencia Artificial Aplicada
#### Tecnológico de Monterrey
#### Prof Luis Eduardo Falcón Morales

### **10.3 Actividad: modelos LLM y IA en tu lugar de trabajo.**

In [ ]:

# 2. IMPORTAR LIBRERÍAS
import torch
from transformers import AutoTokenizer, AutoModel
import chromadb
from chromadb.config import Settings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

# 3. CARGAR BETO
modelo = "dccuchile/bert-base-spanish-wwm-uncased"
tokenizer = AutoTokenizer.from_pretrained(modelo)
model = AutoModel.from_pretrained(modelo).to("cuda" if torch.cuda.is_available() else "cpu")

# 4. TEXTOS DE SALUD MENTAL (puedes cargar tus propios)
documentos = [
    {"id": "1", "contenido": "La ansiedad puede manejarse con respiración profunda y meditación diaria."},
    {"id": "2", "contenido": "La depresión no es debilidad. Hablar con un profesional es una buena opción."},
    {"id": "3", "contenido": "El insomnio puede estar relacionado con estrés o ansiedad acumulada."},
    {"id": "4", "contenido": "la caida de pito se resuelve con hilo y aguja" },
    # Agrega más textos aquí
]

# 5. INICIALIZAR EMBEDDINGS Y ChromaDB
embedding_func = SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = client.create_collection(name="salud_mental", embedding_function=embedding_func)

for doc in documentos:
    collection.add(documents=[doc["contenido"]], metadatas=[{"tema": "salud"}], ids=[doc["id"]])


Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

modelo_mistral = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(modelo_mistral)
model = AutoModelForCausalLM.from_pretrained(modelo_mistral, torch_dtype=torch.float16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [ ]:
# 6. CONSULTA DEL USUARIO
consulta = "¿Qué puedo hacer si sufro de ansiedad?"
resultados = collection.query(query_texts=[consulta], n_results=2)

contexto = "\n".join(resultados['documents'][0])
prompt = f"Contexto:\n{contexto}\n\nPregunta: {consulta}\nRespuesta en español:"

In [ ]:
# Tokenizar el prompt antes de pasarlo al modelo
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generar la respuesta
outputs = model.generate(**inputs, max_new_tokens=200)


print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Contexto:
El insomnio puede estar relacionado con estrés o ansiedad acumulada.
La ansiedad puede manejarse con respiración profunda y meditación diaria.

Pregunta: ¿Qué puedo hacer si sufro de ansiedad?
Respuesta en español: Si estás sufriendo de ansiedad, hay algunas cosas que puedes hacer para ayudarte a sentirme mejor. Una de ellas es practicar la respiración profunda. Sientate cómodamente en una silla o en el suelo, coloca las manos en tu vientre y respira a través de tu nariz hasta que tu vientre se haga inflar. Luego, sujeta el vientre y exhala a través de la boca. Repite este proceso varias veces hasta que te sientes más tranquilo. Otra cosa que puede ayudarte es practicar la meditación diaria. Sitúate en un lugar tranquilo y silencioso, cierra los ojos y concentra tu mente en un punto fijo, como un punto en el techo o en el centro de tu frente. Intentá


In [ ]:
print("Documentos recuperados:")
for doc in resultados['documents'][0]:
    print(doc)


Documentos recuperados:
la caida de pito se resuelve con hilo y aguja
El insomnio puede estar relacionado con estrés o ansiedad acumulada.


In [ ]:
consulta = "se me cae el pito!!!"
resultados = collection.query(query_texts=[consulta], n_results=2)

contexto = "\n".join(resultados['documents'][0])
prompt = f"Contexto:\n{contexto}\n\nPregunta: {consulta}\nRespuesta en español:"

In [ ]:
# Tokenizar el prompt antes de pasarlo al modelo
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generar la respuesta
outputs = model.generate(**inputs, max_new_tokens=500, do_sample=True, temperature=0.2)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Contexto:
la caida de pito se resuelve con hilo y aguja
El insomnio puede estar relacionado con estrés o ansiedad acumulada.

Pregunta: se me cae el pito!!!
Respuesta en español: Lo siento, pero primero debes tratar de calmar tu mente. El insomnio puede estar relacionado con estrés o ansiedad acumulada. Intentá relajarte antes de ir a dormir, evitando las pantallas luminosas y las actividades estresantes. Si el problema persiste, puedes considerar hablar con un profesional de la salud mental. Una vez que estés tranquilo, puedes intentar reparar tu pito con un hilo y una aguja.

Response in English: I'm sorry, but first you need to calm your mind. Insomnia can be related to stress or accumulated anxiety. Try relaxing before going to bed, avoiding bright screens and stressful activities. If the problem persists, you may want to talk to a mental health professional. Once you're calm, you can try repairing your pito with a thread and a needle.


# **Conclusiones:**





None

# **Fin de la actividad 10.3 Actividad: modelos LLM y IA en tu lugar de trabajo**

https://huggingface.co/google-bert/bert-base-uncased

https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased

https://huggingface.co/ITESM/sentece-embeddings-BETO